<a href="https://colab.research.google.com/github/yyyyyokoko/AI-class/blob/master/data_characterization/MAG/Luwei's_MAG_RelatedFieldOfStudy_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Characterization Report for RelatedFieldOfStudy**

In [0]:
#@title
import pandas as pd
import re
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from datetime import date
from google.cloud import bigquery
from IPython.display import display, HTML
from google.colab import auth

auth.authenticate_user()
client = bigquery.Client(project="gcp-cset-projects")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#related = pd.read_gbq("SELECT * FROM gcp_cset_mag.RelatedFieldOfStudy LIMIT 1000", dialect="standard", project_id="gcp-cset-projects")

In [0]:
finalDF = pd.DataFrame(columns=['Colname', 'Has Null?', 'Possible Null Values', '# of Null Values', 'Has Unique Values?', 'Original Data Type', 
                                'Suggested Data Type', 'Note', 'Normalization Ideas'])
finalDF.Colname = ['FieldOfStudyId1', 'Type1', 'FieldOfStudyId2', 'Type2', 'Rank']
finalDF['Possible Null Values'] = ''
finalDF['# of Null Values'] = 0
finalDF['Note'] = ''
finalDF['Normalization Ideas'] = ''  
finalDF['Has Null?'] = 'No'

In [0]:
#check unique values 
def checkUniqueness(colname):
  query = 'SELECT COUNT('+colname+') = COUNT(DISTINCT('+colname+')) as result FROM gcp_cset_mag.RelatedFieldOfStudy'
  result_df = pd.read_gbq(query, dialect="standard", project_id="gcp-cset-projects")
  if result_df['result'][0] == True:
    finalDF.loc[finalDF.Colname == colname, 'Has Unique Values?'] = 'Yes'
    print('Unique: ', colname)
  else:
    finalDF.loc[finalDF.Colname == colname, 'Has Unique Values?'] = 'No'
    query2 = 'SELECT COUNT(CASE WHEN result THEN 1 ELSE 0 END) AS unique FROM (SELECT count('+colname+') as Count, COUNT('+colname+') = 1 AS result FROM gcp_cset_mag.RelatedFieldOfStudy GROUP BY '+colname+' ORDER BY Count DESC)'
    result_df2 = pd.read_gbq(query2, dialect="standard", project_id="gcp-cset-projects")
    print('Number of Unique value in ', colname, ': ', result_df2.unique[0])
  return 

In [5]:
total = pd.read_gbq("select count(*) as total from gcp_cset_mag.RelatedFieldOfStudy", dialect="standard", project_id="gcp-cset-projects")
print('Total number of rows: ', total.total[0])
for i in finalDF.Colname:
  checkUniqueness(i)

Total number of rows:  6690706
Number of Unique value in  FieldOfStudyId1 :  500043
Number of Unique value in  Type1 :  5
Number of Unique value in  FieldOfStudyId2 :  275475
Number of Unique value in  Type2 :  5
Number of Unique value in  Rank :  3037644


In [0]:
#check na
def checknull(colname):
  #check number of NA,TBD,n/a,null,none,TBA
  print('Checking ', colname)
  lst = ['na', 'NA', 'tbd', 'TBD', 'n/a', 'N/A', 'Null', 'null', 'NULL', 'none', 'None', 'NONE', 'tba', 'TBA']
  for i in lst:
    sql_string_build = 'select count(' + colname + ') as Count from gcp_cset_mag.RelatedFieldOfStudy where regexp_contains( CAST(' + colname+ ' AS STRING), r\"' + i + '^\s*$\") or CAST('+colname+' AS STRING)=\"' + i + '\"' 
    temp_df = pd.read_gbq(sql_string_build, dialect="standard", project_id="gcp-cset-projects")
    if temp_df['Count'][0] > 0:
      finalDF.loc[finalDF.Colname == colname, 'Has Null?'] = 'Yes'
      finalDF.loc[finalDF.Colname == colname, 'Possible Null Values'] = finalDF.loc[finalDF.Colname == colname, 'Possible Null Values'] +' ' + i
      finalDF.loc[finalDF.Colname == colname, '# of Null Values'] += temp_df['Count'][0]
  #check empty string
  sql_string_build_2 = "select count(" + colname + ") as Count from gcp_cset_mag.RelatedFieldOfStudy where regexp_contains( CAST(" + colname+ " AS STRING) , r\"^\s*$\")"
  temp_df_2 = pd.read_gbq(sql_string_build_2, dialect="standard", project_id="gcp-cset-projects")
  if temp_df_2['Count'][0] > 0:
    finalDF.loc[finalDF.Colname == colname, 'Has Null?'] = 'Yes'
    finalDF.loc[finalDF.Colname == colname, 'Possible Null Values'] = finalDF.loc[finalDF.Colname == colname, 'Possible Null Values'] +' empty string'
    finalDF.loc[finalDF.Colname == colname, '# of Null Values'] += temp_df_2['Count'][0]
  
  return 

In [9]:
for i in finalDF.Colname:
  checknull(i)

Checking  FieldOfStudyId1
Checking  Type1
Checking  FieldOfStudyId2
Checking  Type2
Checking  Rank


In [0]:
finalDF['Original Data Type'] = ['INTEGER', 'STRING', 'INTEGER', 'STRING', 'FLOAT']   
finalDF['Suggested Data Type'] = ['INTEGER', 'STRING', 'INTEGER', 'STRING', 'FLOAT']    
# finalDF.loc[finalDF['Colname'] == 'DisplayName', 'Note'] = 'Close to unique'
# finalDF.loc[finalDF['Colname'] == 'NormalizedName', 'Note'] = 'Close to unique'

## **Report Summary**

In [11]:
finalDF

,Colname,Has Null?,Possible Null Values,# of Null Values,Has Unique Values?,Original Data Type,Suggested Data Type,Note,Normalization Ideas
0,FieldOfStudyId1,No,,0,No,INTEGER,INTEGER,,
1,Type1,No,,0,No,STRING,STRING,,
2,FieldOfStudyId2,No,,0,No,INTEGER,INTEGER,,
3,Type2,No,,0,No,STRING,STRING,,
4,Rank,No,,0,No,FLOAT,FLOAT,,


1. From the random sample of **Rank** attribute, we noticed that while most data values are between 0 and 4, there are some extremely large values in the data. This attribute may need to be further investigated and normalized. 

## **Graphs**

Due to the size of the dataset, some graphs only applied randomly sampled data.

In [103]:
#field of study id 1
temp = pd.read_gbq('select FieldOfStudyId1, COUNT(FieldOfStudyId1) as count from gcp_cset_mag.RelatedFieldOfStudy GROUP BY FieldOfStudyId1 ORDER BY count DESC LIMIT 20', dialect="standard", project_id="gcp-cset-projects")
temp['FieldOfStudyId1'] = temp['FieldOfStudyId1'].astype(str)
fig = px.bar(temp, x='FieldOfStudyId1', y='count', text='count', title='Bar chart of 15 Most Frequent FieldOfStudyId1')
fig.update_layout(xaxis={'type': 'category'})
fig.show()

In [104]:
#Type1
temp = pd.read_gbq('select Type1, COUNT(Type1) as count from gcp_cset_mag.RelatedFieldOfStudy GROUP BY Type1 ORDER BY count DESC LIMIT 20', dialect="standard", project_id="gcp-cset-projects")
temp['Type1'] = temp['Type1'].astype(str)
fig = px.bar(temp, x='Type1', y='count', text='count', title='Bar chart of Type1')
fig.update_layout(xaxis={'type': 'category'})
fig.show()

In [105]:
#field of study id 2
temp = pd.read_gbq('select FieldOfStudyId2, COUNT(FieldOfStudyId2) as count from gcp_cset_mag.RelatedFieldOfStudy GROUP BY FieldOfStudyId2 ORDER BY count DESC LIMIT 20', dialect="standard", project_id="gcp-cset-projects")
temp['FieldOfStudyId2'] = temp['FieldOfStudyId2'].astype(str)
fig = px.bar(temp, x='FieldOfStudyId2', y='count', text='count', title='Bar chart of 15 Most Frequent FieldOfStudyId2')
fig.update_layout(xaxis={'type': 'category'})
fig.show()

In [106]:
#Type2
temp = pd.read_gbq('select Type2, COUNT(Type2) as count from gcp_cset_mag.RelatedFieldOfStudy GROUP BY Type2 ORDER BY count DESC LIMIT 20', dialect="standard", project_id="gcp-cset-projects")
temp['Type2'] = temp['Type2'].astype(str)
fig = px.bar(temp, x='Type2', y='count', text='count', title='Bar chart of Type2')
fig.update_layout(xaxis={'type': 'category'})
fig.show()

In [117]:
#type 1 + type 2, with average rank
temp = pd.read_gbq('SELECT Type1, Type2, COUNT(*) as count, AVG(Rank) as avg_rank from gcp_cset_mag.RelatedFieldOfStudy GROUP BY Type1, Type2 ORDER BY count DESC LIMIT 20', dialect="standard", project_id="gcp-cset-projects")
temp['text'] = temp['Type1'] + '+' + temp['Type2']
fig = px.bar(temp, x='text', y='count', text='count', title='Bar chart of Type1 + Type2', hover_data=['avg_rank'])
fig.update_layout(xaxis={'type': 'category'})
fig.show()

In [118]:
#rank, random selected the 1% of data for the histogram
temp = pd.read_gbq('SELECT Rank FROM gcp_cset_mag.RelatedFieldOfStudy ORDER BY RAND() LIMIT 66907', dialect="standard", project_id="gcp-cset-projects")
fig = px.histogram(temp, x="Rank", title='Histogram of Rank, randomly sampled')
fig.show()


In [121]:
temp = pd.read_gbq('SELECT * FROM gcp_cset_mag.RelatedFieldOfStudy ORDER BY Rank DESC LIMIT 20', dialect="standard", project_id="gcp-cset-projects")
temp

,FieldOfStudyId1,Type1,FieldOfStudyId2,Type2,Rank,import_time
0,199733313,general,160443848,general,9055.56200,2020-06-06 13:46:38
1,160443848,general,199733313,general,9055.56200,2020-06-06 13:46:38
2,160443848,general,2775868463,general,8146.71143,2020-06-06 13:46:38
3,2775868463,general,160443848,general,8146.71143,2020-06-06 13:46:38
4,52130261,general,2775868463,general,7217.41748,2020-06-06 13:46:38
5,2775868463,general,52130261,general,7217.41748,2020-06-06 13:46:38
6,52130261,general,160443848,general,7192.82031,2020-06-06 13:46:38
7,160443848,general,52130261,general,7192.82031,2020-06-06 13:46:38
8,2775868463,general,199733313,general,6654.98200,2020-06-06 13:46:38
9,199733313,general,2775868463,general,6654.98200,2020-06-06 13:46:38
